# Sensitivity analysis of the critical head difference

In this example, we will demonstrate how to conduct a sensitivity analysis of the critical head difference model developed by Sellmeijer. This model is applicable to the piping failure mechanism, which addresses backward internal erosion beneath dikes with predominantly horizontal seepage paths.

### Define model

First, let's import the necessary packages:

In [29]:
from ptk import *
import matplotlib.pyplot as plt
import numpy as np

The critical head difference, $H_c$, according to Sellmeijer's model is described by the following equations:

$F_{resistance}=\eta\cdot \frac{\gamma_{sub,particles}}{\gamma_{water}}\cdot \tan \theta_{sellmeijer,rev}$


$F_{scale}=\frac{d_{70.m}}{\sqrt[3]{\kappa\cdot L}}\cdot\left(\frac{d_{70}}{d_{70.m}}\right)^{0.4}$ and $\kappa = \frac{\nu_{water}}{g}\cdot k$


$F_{geometry}=0.91\cdot \left(\frac{D}{L}\right)^{\frac{0.28}{\left(\frac{D}{L}\right)^{2.8}-1}+0.04}$

$H_c = F_{resistance} \cdot F_{scale} \cdot F_{geometry} \cdot L$

where: <br>
$L$ - seepage length (m) <br>
$D$ - thickness of upper sand layer (m) <br>
$\theta$ - bedding angle ($\circ$) <br>
$d_{70}$ - particle diameter (m) <br>
$k$ - permeability of the upper sand layer (m/s)

In [30]:
def model_sellmeijer(theta, k, L, d70, D):

    RD = 0.725
    RDm = 0.725
    d70m = 2.08e-4
    nu = 1.33e-6
    eta = 0.25

    F_resistance = 1.65*eta*np.tan(theta/180*np.pi)*(RD/RDm)**0.35
    F_scale = d70m/(nu/9.81*k*L)**(1/3)*(d70/d70m)**0.39
    F_geometry = 0.91*(D/L)**(0.28/(((D/L)**2.8)-1)+0.04)

    if D==L:
        F_geometry = 1.0
    
    delta_h_c = F_resistance * F_scale * F_geometry * L

    return delta_h_c

### Perform sensitivity analysis

To conduct a sensitivity analysis, we begin by creating a sensitivity project and defining the model:

In [31]:
project = SensitivityProject()
project.model = model_sellmeijer

First, we define the all input parameters of the model as deterministic variables:

In [33]:
project.variables["theta"].distribution = "deterministic"
project.variables["theta"].mean = 37.0

project.variables["k"].distribution = "deterministic"
project.variables["k"].mean = 0.000245598

project.variables["L"].distribution = "deterministic"
project.variables["L"].mean = 40.0

project.variables["d70"].distribution = "deterministic"
project.variables["d70"].mean = 0.00019

project.variables["D"].distribution = "deterministic"
project.variables["D"].mean = 30.0


We start with the method `crude_monte_carlo`. We choose the calculation settings: `minimum_directions` and `maximum_directions`. The reliability analysis is executed using `project.run()`, and the results are accessed from `project.stochast`.

In [1]:
project.settings.sensitivity_method = "crude_monte_carlo"
project.settings.minimum_directions = 100
project.settings.maximum_directions = 200
effect_low = []; effect_high = []

for var in ["theta", "k", "L", "d70", "D"]:
        
        current_value = project.variables[var].mean
        project.variables[var].distribution = "uniform"
        project.variables[var].minimum = current_value*0.25
        project.variables[var].maximum = current_value*1.25
    
        project.run()
        sens = project.stochast

        print(f"Effect {var}:({sens.minimum}, {sens.maximum})")
        
        project.variables[var].distribution = "deterministic"

NameError: name 'project' is not defined